The Perceptron

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()

X = iris.data[:, (2, 3)] # petal length and petal width
y = (iris.target == 0).astype(np.int)

per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

X_pred = per_clf.predict([[2, 0.5]])
print(X_pred)

[1]


/home/dimension/.virtualenvs/data-science/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [2]:
import os
from six.moves import urllib
from scipy.io import loadmat

mnist_path = os.path.join("/home/dimension/", "scikit_learn_data/mldata", "mnist-original.mat")

if not os.path.isfile(mnist_path):
    # # download dataset from github.
    mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, "wb") as f:
        content = response.read()
        f.write(content)

mnist_raw = loadmat(mnist_path)
mnist = {
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0],
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
}
print("Done!")

Done!


Training an MLP with TensorFlow's High-Level API

In [6]:
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

X, y = mnist["data"], mnist["target"]

scaler = StandardScaler()
scaler.fit(X)
y = y.astype(np.int)

# MNIST dataset is actually already split into training(first 60,000 images) and test set (10,000 images)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:] 

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10, feature_columns=feature_cols)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=4000)

y_pred = list(dnn_clf.predict(X_test))
accuracy_score(y_test, y_pred)


/home/dimension/.virtualenvs/data-science/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpow_e53kq', '_log_step_count_steps': 100, '_num_worker_replicas': 0, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2422f85a90>, '_tf_random_seed': None, '_train_distribute': None, '_master': '', '_keep_checkpoint_max': 5, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_evaluation_master': '', '_task_type': None, '_environment': 'local', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_task_id': 0, '_session_config': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpow_e53kq/model.ckpt.
INFO:tensorflow:step = 1, loss = 131.1536
INFO:tensorfl

0.914

Training a DNN using plain TensorFlow

Construction Phase

In [38]:
tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

Creating Neural Network

In [39]:
def neuron_layer(input_X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(input_X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(input_X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

Creating Neural Network using TensorFlow Library

In [32]:
with tf.name_scope("dnn_tf"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

Calculating Loss

In [40]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

Training Model

In [41]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Execution Phase

In [42]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [43]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iterations in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
        
        print("Epoch: ", epoch, " Train Accuracy: ", acc_train, " Test Accuracy: ", acc_test)
    
    save_path = saver.save(sess, "./my_model_final.ckpt")

Epoch:  0  Train Accuracy:  0.92  Test Accuracy:  0.9144
Epoch:  1  Train Accuracy:  0.96  Test Accuracy:  0.9329
Epoch:  2  Train Accuracy:  0.94  Test Accuracy:  0.9395
Epoch:  3  Train Accuracy:  0.9  Test Accuracy:  0.9464
Epoch:  4  Train Accuracy:  0.98  Test Accuracy:  0.9504
Epoch:  5  Train Accuracy:  0.96  Test Accuracy:  0.9553
Epoch:  6  Train Accuracy:  0.98  Test Accuracy:  0.9582
Epoch:  7  Train Accuracy:  0.94  Test Accuracy:  0.9602
Epoch:  8  Train Accuracy:  0.96  Test Accuracy:  0.9629
Epoch:  9  Train Accuracy:  0.98  Test Accuracy:  0.9637
Epoch:  10  Train Accuracy:  0.98  Test Accuracy:  0.9646
Epoch:  11  Train Accuracy:  0.98  Test Accuracy:  0.966
Epoch:  12  Train Accuracy:  0.96  Test Accuracy:  0.9674
Epoch:  13  Train Accuracy:  0.98  Test Accuracy:  0.9686
Epoch:  14  Train Accuracy:  0.98  Test Accuracy:  0.9693
Epoch:  15  Train Accuracy:  0.98  Test Accuracy:  0.9705
Epoch:  16  Train Accuracy:  1.0  Test Accuracy:  0.9701
Epoch:  17  Train Accuracy: